### Projeto simples de um sostema bancario 

--> funções principais:
    Sacar com limite de 3 saques diarios, cada saque de no máximo 500 reais
    depositar 
    Extrato --> com todas movimentações.

In [7]:
import textwrap # Para formatação de texto
from datetime import datetime # Para manipulação de data e hora
import os # Para limpar a tela do terminal
import json # Para manipulação de arquivos JSON
import locale # Para formatação de números (opcional, dependendo do sistema)
from num2words import num2words # Para converter números em palavras (opcional)

def menu():
    """Exibe o menu de opções para o usuário."""
    menu_texto = """\n
    ================ MENU ================
    [c]\tCriar Cliente
    [n]\tCriar Conta
    [l]\tListar Contas
    [d]\tDepositar
    [s]\tSacar
    [e]\tExtrato
    [q]\tSair
    => """
    return input(textwrap.dedent(menu_texto))

def criar_cliente(clientes):
    """Cria um novo cliente e o armazena na lista de clientes."""
    cpf = input("Informe o CPF (somente números): ")
    cliente = filtrar_cliente(cpf, clientes)

    if cliente:
        print("\n❌ Já existe um cliente com este CPF!")
        return

    nome = input("Informe o nome completo: ")
    data_nascimento = input("Informe a data de nascimento (dd-mm-aaaa): ")
    endereco = input("Informe o endereço (logradouro, nro - bairro - cidade/sigla estado): ")

    clientes.append({"nome": nome, "data_nascimento": data_nascimento, "cpf": cpf, "endereco": endereco})
    print("\n✅ Cliente criado com sucesso!")

def filtrar_cliente(cpf, clientes):
    """Busca um cliente na lista pelo CPF."""
    clientes_filtrados = [cliente for cliente in clientes if cliente["cpf"] == cpf]
    return clientes_filtrados[0] if clientes_filtrados else None

def criar_conta(agencia, numero_conta, clientes, contas):
    """Cria uma nova conta bancária vinculada a um cliente."""
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if cliente:
        contas.append({"agencia": agencia, "numero_conta": numero_conta, "cliente": cliente, "saldo": 0, "extrato": "", "numero_saques": 0})
        print("\n✅ Conta criada com sucesso!")
        return numero_conta + 1
    
    print("\n❌ Cliente não encontrado. Fluxo de criação de conta encerrado!")
    return numero_conta

def listar_contas(contas):
    """Exibe todas as contas cadastradas."""
    if not contas:
        print("\nNenhuma conta cadastrada.")
        return

    print("\n================ LISTA DE CONTAS ================")
    for conta in contas:
        linha = f"""\
            Agência:\t{conta['agencia']}
            C/C:\t\t{conta['numero_conta']}
            Titular:\t{conta['cliente']['nome']}
        """
        print(textwrap.dedent(linha))
        print("-" * 45)

def depositar(clientes, contas):
    """Realiza um depósito em uma conta específica do cliente."""
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\n❌ Cliente não encontrado!")
        return

    conta = selecionar_conta_cliente(cliente, contas)
    if not conta:
        return

    try:
        valor = float(input("Informe o valor do depósito: R$ "))
        if valor > 0:
            conta["saldo"] += valor
            timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            
            # --- Bloco de formatação e registro ---
            valor_formatado = locale.currency(valor, grouping=True)
            valor_extenso = num2words(valor, lang='pt_BR', to='currency')
            
            registro_extrato = (
                f"[{timestamp}] Depósito:\t{valor_formatado}\n"
                f"\t\t({valor_extenso.capitalize()})\n"
            )
            conta["extrato"] += registro_extrato
            # --- Fim do Bloco ---
            
            print("\n✅ Depósito realizado com sucesso!")
        else:
            print("\n❌ Operação falhou! O valor informado é inválido.")
    except ValueError:
        print("\n❌ Entrada inválida. Por favor, informe um valor numérico.")

def sacar(clientes, contas):
    """Realiza um saque de uma conta específica do cliente."""
    LIMITE_SAQUES = 3
    LIMITE_VALOR_SAQUE = 500

    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\n❌ Cliente não encontrado!")
        return
    
    conta = selecionar_conta_cliente(cliente, contas)
    if not conta:
        return

    try:
        valor = float(input("Informe o valor do saque: R$ "))

        # ... (as validações de saque continuam iguais)
        if conta["numero_saques"] >= LIMITE_SAQUES:
            print("\n❌ Operação falhou! Número máximo de saques diários excedido.")
        elif valor > conta["saldo"]:
            print("\n❌ Operação falhou! Saldo insuficiente.")
        elif valor > LIMITE_VALOR_SAQUE:
            print(f"\n❌ Operação falhou! O valor do saque excede o limite de R$ {LIMITE_VALOR_SAQUE:.2f}.")
        elif valor > 0:
            conta["saldo"] -= valor
            conta["numero_saques"] += 1
            timestamp = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
            
            # --- Bloco de formatação e registro ---
            valor_formatado = locale.currency(valor, grouping=True)
            valor_extenso = num2words(valor, lang='pt_BR', to='currency')
            
            registro_extrato = (
                f"[{timestamp}] Saque:\t\t{valor_formatado}\n"
                f"\t\t({valor_extenso.capitalize()})\n"
            )
            conta["extrato"] += registro_extrato
            # --- Fim do Bloco ---
            
            print("\n✅ Saque realizado com sucesso!")
        else:
            print("\n❌ Operação falhou! O valor informado é inválido.")
    except ValueError:
        print("\n❌ Entrada inválida. Por favor, informe um valor numérico.")

def mostrar_extrato(clientes, contas):
    """Exibe o extrato de uma conta específica do cliente."""
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\n❌ Cliente não encontrado!")
        return

    conta = selecionar_conta_cliente(cliente, contas)
    if not conta:
        return

    print("\n================ EXTRATO ================")
    print(f"Titular: {conta['cliente']['nome']}")
    print(f"Agência: {conta['agencia']} | C/C: {conta['numero_conta']}")
    print("-" * 40)
    
    print("Não foram realizadas movimentações." if not conta["extrato"] else conta["extrato"])
    
    # --- MUDANÇA APLICADA AQUI ---
    # Formata o saldo final com valor numérico e por extenso
    saldo_formatado = locale.currency(conta['saldo'], grouping=True)
    saldo_extenso = num2words(conta['saldo'], lang='pt_BR', to='currency')

    print(f"\nSaldo:\t\t{saldo_formatado}")
    print(f"\t\t({saldo_extenso.capitalize()})")
    print("==========================================")

def limpar_tela():
    """Limpa o console do terminal."""
    os.system('cls' if os.name == 'nt' else 'clear')

def salvar_dados(clientes, contas):
    """Salva os dados de clientes e contas em um arquivo JSON."""
    dados = {"clientes": clientes, "contas": contas}
    try:
        with open("banco_dados.json", "w", encoding="utf-8") as f:
            json.dump(dados, f, indent=4)
        # print("Debug: Dados salvos com sucesso.") # Opcional: para depuração
    except IOError as e:
        print(f"\n❌ Erro ao salvar os dados: {e}")

def carregar_dados():
    """Carrega os dados de clientes e contas de um arquivo JSON."""
    try:
        with open("banco_dados.json", "r", encoding="utf-8") as f:
            dados = json.load(f)
            return dados.get("clientes", []), dados.get("contas", [])
    except FileNotFoundError:
        # Se o arquivo não existe (primeira execução), retorna listas vazias
        return [], []
    except json.JSONDecodeError:
        # Se o arquivo está vazio ou corrompido, retorna listas vazias
        print("\n⚠️ Arquivo de dados encontrado, mas está vazio ou corrompido. Começando do zero.")
        return [], []    

def selecionar_conta_cliente(cliente, contas):
    """
    Permite que o usuário selecione uma conta dentre as suas.
    Retorna a conta selecionada ou None se nenhuma for selecionada.
    """
    contas_cliente = [c for c in contas if c['cliente']['cpf'] == cliente['cpf']]

    if not contas_cliente:
        print("\n❌ Este cliente não possui contas cadastradas.")
        return None

    # Seleciona automaticamente se houver apenas uma conta
    if len(contas_cliente) == 1:
        conta = contas_cliente[0]
        print(f"\n✅ Conta C/C {conta['numero_conta']} selecionada automaticamente.")
        return conta

    # Pede para o usuário escolher se houver múltiplas contas
    print("\nEste cliente possui mais de uma conta. Por favor, selecione uma:")
    for i, conta in enumerate(contas_cliente):
        print(f"  [{i + 1}] Agência: {conta['agencia']}, C/C: {conta['numero_conta']}")

    try:
        escolha = int(input("Digite o número da opção desejada: "))
        # Valida se a escolha está dentro do intervalo da lista
        if 1 <= escolha <= len(contas_cliente):
            return contas_cliente[escolha - 1] # Ajusta o índice (lista começa em 0)
        else:
            print("\n❌ Opção inválida. Por favor, tente novamente.")
            return None
    except ValueError:
        print("\n❌ Entrada inválida. Por favor, digite um número.")
        return None    

def main():

    """Função principal que executa o sistema bancário."""
    try:
        locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    except locale.Error:
        print("Atenção: A localidade 'pt_BR.UTF-8' não está disponível...")

    AGENCIA = "0001"
    
    # Carrega os dados do arquivo ao iniciar
    clientes, contas = carregar_dados()
    
    # Define o próximo número de conta baseado nas contas já existentes
    if contas:
        numero_conta_atual = max(c['numero_conta'] for c in contas) + 1
    else:
        numero_conta_atual = 1

    while True:
        limpar_tela()
        opcao = menu()

        if opcao == "c":
            criar_cliente(clientes)
            salvar_dados(clientes, contas) # Salva após a alteração
            input("\nPressione Enter para continuar...")
        
        elif opcao == "n":
            numero_conta_atual = criar_conta(AGENCIA, numero_conta_atual, clientes, contas)
            salvar_dados(clientes, contas) # Salva após a alteração
            input("\nPressione Enter para continuar...")

        elif opcao == "l":
            listar_contas(contas)
            input("\nPressione Enter para continuar...")

        elif opcao == "d":
            depositar(clientes, contas)
            salvar_dados(clientes, contas) # Salva após a alteração
            input("\nPressione Enter para continuar...")

        elif opcao == "s":
            sacar(clientes, contas)
            salvar_dados(clientes, contas) # Salva após a alteração
            input("\nPressione Enter para continuar...")

        elif opcao == "e":
            mostrar_extrato(clientes, contas)
            input("\nPressione Enter para continuar...")

        elif opcao == "q":
            limpar_tela()
            print("\nSaindo do sistema... Obrigado por usar nosso banco!\n")
            break

        else:
            print("\n❌ Operação inválida, por favor selecione novamente a operação desejada.")
            input("\nPressione Enter para continuar...")


if __name__ == "__main__":
    main()


✅ Conta C/C 1 selecionada automaticamente.

================ EXTRATO ================
Titular: Kaique Santo
Agência: 0001 | C/C: 1
----------------------------------------
[16-08-2025 13:44:00] Depósito:	R$ 100000000.00
[16-08-2025 14:00:24] Depósito:	R$ 150.000.000.000,00
		(Cento e cinquenta bilhões de reais)


Saldo:		R$ 150.100.000.000,00
		(Cento e cinquenta bilhões e cem milhões de reais)

Saindo do sistema... Obrigado por usar nosso banco!

